<a href="https://colab.research.google.com/github/y-arjun-y/cloudclassifier/blob/main/cloudclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Cloud Classifier Project for fast.ai - Practical Deep Learning for Coders - Lesson 2](https://github.com/y-arjun-y/cloudclassifier)

### Download dataset (if you don't have a Bing Image Search API Key)

In [ ]:
!npx degit y-arjun-y/cloudclassifier -f

### Imports (it's normal for it to request access to GDrive for Desktop)

In [ ]:
!pip install fastbook==0.0.17
!pip uninstall tornado -y
!yes | pip install tornado==5.1.0

import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

### Bing Image Search API (optional if you downloaded the dataset from the repository)

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', 'BING_IMAGE_SEARCH_API_KEY')

In [ ]:
# Using only three basic cloud types. Feel free to download more!
cloud_types = 'cumulus cloud', 'stratus cloud', 'cirrus cloud'
path = Path('clouds')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in cloud_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o}')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

(#3) [None,None,None]

### DataLoaders and ResNet-18

In [ ]:
clouds = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [ ]:
clouds = clouds.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms()
)
dls = clouds.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,1.541311,0.781851,0.302326,00:39


epoch,train_loss,valid_loss,error_rate,time
0,1.089366,0.715509,0.209302,00:39
1,1.046867,0.646763,0.174419,00:40
2,0.884108,0.630488,0.139535,00:40
3,0.841090,0.614772,0.151163,00:39


### Exporting the model (you need to run this)

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

### Supplying your own cloud image and getting a prediction!

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()

In [ ]:
pred, pred_idx, probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

Label(value='Prediction: cirrus cloud; Probability: 0.9998')